In [3]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [5]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [6]:
df.drop(['packer_type'],axis=1,inplace=True)

In [7]:
from sklearn.utils import shuffle
df = shuffle(df)

In [8]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [9]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [10]:
df.drop(['index'],axis=1,inplace=True)

In [11]:
df_5percent=df[0:4949]

In [12]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,208,4,1,0,1,...,1,1,3,0,6.013823,6.159066,90112,5.648154,1,1
1,144,3,4,65535,184,240,3,1,1,1,...,1,0,3,0,3.617785,0.000000,121856,5.521669,0,1
2,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.492561,0.000000,126976,5.219446,0,0
3,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0
4,144,3,4,65535,184,200,3,1,1,1,...,1,0,3,0,6.926852,1.894808,88576,6.838387,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4944,144,3,4,65535,184,248,4,1,1,1,...,1,0,4,0,3.782408,0.000000,125440,5.336721,0,1
4945,144,3,4,65535,184,248,4,1,1,1,...,1,2,2,0,0.000000,0.255780,118784,5.489687,0,1
4946,144,3,4,65535,184,208,10,1,1,1,...,1,6,4,0,7.688110,0.000000,417280,7.971555,0,1
4947,144,3,4,65535,184,192,2,1,0,1,...,1,1,1,0,0.000000,0.000000,16456,5.956983,1,0


In [13]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2360, 69)
class 1: (2589, 69)


In [14]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [15]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     208.0               4.0   
1      144.0   3.0        4.0     65535.0  184.0     240.0               3.0   
2      144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
3      144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
4      144.0   3.0        4.0     65535.0  184.0     200.0               3.0   
...      ...   ...        ...         ...    ...       ...               ...   
4944   144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
4945   144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
4946   144.0   3.0        4.0     65535.0  184.0     208.0              10.0   
4947   144.0   3.0        4.0     65535.0  184.0     192.0               2.0   
4948   144.0   3.0        4.0     65535.0  184.0     184.0               3.0   

      C

In [16]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4949.0,...,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4949.000000,4.949000e+03,4949.000000,4949.000000,4949.000000
mean,153.273793,11.038796,9.162659,65129.903213,203.545363,222.979188,4.663770,0.990503,0.362902,1.0,...,0.998990,1.355425,3.308345,0.155385,4.946466,2.522564,7.684141e+05,6.367944,0.543544,0.523136
std,632.534403,402.966256,258.235211,5050.931411,993.829846,48.327456,1.928518,0.096998,0.480885,0.0,...,0.031772,1.627733,1.146456,0.362308,2.518281,2.687384,5.123353e+06,1.118806,0.498151,0.499515
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.886006,0.000000,6.152400e+04,5.696312,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.152807,1.551060,1.217040e+05,6.391190,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.507302,4.679578,3.031360e+05,7.328246,1.000000,1.000000
max,37008.000000,20050.000000,12851.000000,65535.000000,65534.000000,648.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999131,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [17]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [18]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [19]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [20]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      51.25
0.0      48.75
dtype: float64

In [21]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      60.0
0.0      40.0
dtype: float64

In [22]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [23]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [24]:
circuit(weights_init, data[0])

tensor(-0.04159936, requires_grad=True)

In [25]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [26]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [27]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [28]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [29]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(0.32498599, requires_grad=False), tensor(-0.23123358, requires_grad=False)], Y = -1
X = [tensor(-0.11383535, requires_grad=False), tensor(-0.25405911, requires_grad=False)], Y =  1
X = [tensor(-0.11504802, requires_grad=False), tensor(-0.34851039, requires_grad=False)], Y = -1
X = [tensor(0.13861093, requires_grad=False), tensor(-0.24560102, requires_grad=False)], Y = -1
X = [tensor(-0.15371253, requires_grad=False), tensor(3.3776994, requires_grad=False)], Y =  1


In [30]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [31]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0992333 | Accuracy: 0.4875000 
Iter:     2 | Cost: 1.6377753 | Accuracy: 0.4875000 
Iter:     3 | Cost: 1.2634952 | Accuracy: 0.4875000 
Iter:     4 | Cost: 1.0198334 | Accuracy: 0.4875000 
New best
Iter:     5 | Cost: 1.0033550 | Accuracy: 0.5125000 
New best
Iter:     6 | Cost: 1.0037496 | Accuracy: 0.5375000 
New best
Iter:     7 | Cost: 0.9785500 | Accuracy: 0.5687500 
Iter:     8 | Cost: 0.9878813 | Accuracy: 0.5187500 
Iter:     9 | Cost: 0.9668441 | Accuracy: 0.5500000 
New best
Iter:    10 | Cost: 0.9318536 | Accuracy: 0.5875000 
Iter:    11 | Cost: 0.9231797 | Accuracy: 0.5875000 
Iter:    12 | Cost: 0.9268597 | Accuracy: 0.5750000 
New best
Iter:    13 | Cost: 0.9103820 | Accuracy: 0.6937500 
Iter:    14 | Cost: 0.9215186 | Accuracy: 0.5750000 
Iter:    15 | Cost: 0.9702879 | Accuracy: 0.5562500 
Iter:    16 | Cost: 1.0178798 | Accuracy: 0.5562500 
Iter:    17 | Cost: 1.0816225 | Accuracy: 0.5562500 
Iter:    18 | Cost: 1.0941425 | Accuracy: 0.5

In [32]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [33]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.2475420975962987, Accuracy: 55.00000000000001%


In [34]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,1.0
2,1.0,-1.0
3,1.0,-1.0
4,-1.0,1.0
5,1.0,-1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,-1.0
9,1.0,1.0


In [35]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         2
         1.0       0.92      0.58      0.71        38

    accuracy                           0.55        40
   macro avg       0.46      0.29      0.35        40
weighted avg       0.87      0.55      0.67        40

0.9166666666666666
0.5789473684210527
0.7096774193548387
0.2894736842105263
